In [3]:
import numpy as np
import pandas as pd
import itertools

from scipy import stats
from statsmodels.stats.descriptivestats import sign_test
from statsmodels.stats.weightstats import zconfint

In [4]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [5]:
data = np.array([49,58,75,110,112,132,151,276,281,362])

In [6]:
stats.wilcoxon(data-200)

WilcoxonResult(statistic=17.0, pvalue=0.28450269791120752)

In [7]:
#Q4 0.2845

In [8]:
d1 = np.array([22,22,15,13,19,19,18,20,21,13,13,15])
d2 = np.array([17,18,18,15,12,4,14,15,10])

In [9]:
stats.mannwhitneyu(d1,d2,alternative='greater')

MannwhitneyuResult(statistic=81.0, pvalue=0.029004992720873729)

In [10]:
#Q5 = 0.029

In [11]:
## Bootstrap

In [12]:
data = pd.read_csv('challenger.txt', '\t')

In [14]:
data.head()

,Unnamed: 0,Temperature,Incident
0,Apr12.81,18.9,0
1,Nov12.81,21.1,1
2,Mar22.82,20.6,0
3,Nov11.82,20.0,0
4,Apr04.83,19.4,0


In [22]:
def get_bootstrap_samples(data, n_samples):
    indices = np.random.randint(0, len(data), (n_samples, len(data)))
    samples = data[indices]
    return samples

In [23]:
def stat_intervals(stat, alpha):
    boundaries = np.percentile(stat, [100 * alpha / 2., 100 * (1 - alpha / 2.)])
    return boundaries

In [24]:
neg_temp = data[data.Incident == 0].Temperature.values
pos_temp = data[data.Incident == 1].Temperature.values

In [27]:
np.random.seed(0)

neg_temp_scores = np.array(list(map(np.mean, get_bootstrap_samples(neg_temp, 1000))))
pos_temp_scores = np.array(list(map(np.mean, get_bootstrap_samples(pos_temp, 1000))))

means_diff = neg_temp_scores - pos_temp_scores

In [30]:
print("95% confidence interval for the incidents: ",  stat_intervals(means_diff, 0.05))

95% confidence interval for the incidents:  [ 1.42299107  7.93861607]


In [ ]:
#Q6 - 1.423

In [31]:
## Перестановочный критерий

In [32]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

In [39]:
def get_random_combinations(n1, n2, max_combinations):
    index = list(range(n1 + n2))
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

In [40]:
def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), filter(lambda i: i not in index, range(n))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

In [41]:
def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [43]:
np.random.seed(0)

In [44]:
print("p-value: %f" % permutation_test(neg_temp, pos_temp, max_permutations = 10000))

p-value: 0.007000
